In [7]:
%load_ext autoreload
%autoreload 2

import os
os.chdir('/home/jk394348/hyper-dl')
print(os.getcwd())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/jk394348/hyper-dl


In [8]:
import hydra
from hydra.utils import get_original_cwd
from hydra import initialize, compose

overrides = [
    'epochs=5'
]

with initialize(config_path="../conf"):
    cfg = compose(config_name="config.yaml", overrides=overrides)

/home/jk394348/hyper-dl


In [10]:
import pickle
from omegaconf import DictConfig
from config import run_training
from data import build_loaders
from plot import setup_dirs, dump_results


def build_dataset_path(seed, n_samples, dim, eps, curv, transform_dim):
    template = 'dim={},n_samples={},eps={},transform_dim={},curv={},seed={}.pkl'
    return template.format(int(dim), int(n_samples), eps, transform_dim, curv, seed)

Working directory : /home/jk394348/hyper-dl
https://app.neptune.ai/jkociniak/hyperbolic-dist-pred/e/HYPERDIST-2717
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Found device: cuda
[1] train loss (mean of batch losses): 1.7771011402675083
[1] val loss: 0.7505387329101563
[2] train loss (mean of batch losses): 0.3819587475367955
[2] val loss: 0.22569158058166505
[3] train loss (mean of batch losses): 0.18705775933946883
[3] val loss: 0.1722919038772583
[4] train loss (mean of batch losses): 0.16011666826520649
[4] val loss: 0.15903715333938598
[5] train loss (mean of batch losses): 0.15092442943028042
[5] val loss: 0.15443335494995117
Training finished
Generating results...
Experiment finished
Shutting down background jobs, please wait a moment...
Done!


Waiting for the remaining 21 operations to synchronize with Neptune. Do not kill this process.


All 21 operations synced, thanks for waiting!


In [4]:
cwd = os.getcwd()
print("Working directory : {}".format(cwd))

dataset_path = build_dataset_path(**cfg['dataset_params'])
dataset_path = os.path.join(cwd, 'datasets', dataset_path)

with open(dataset_path, 'rb') as f:
    datasets = pickle.load(f)
loaders = build_loaders(datasets, cfg['bs'], cfg['num_workers'])

results, run = run_training(loaders=loaders,
                            cfg=cfg,
                            **cfg)

name: DoubleInputHyperbolicFFN
input_dim: 2
hidden_dims: []
output_dim: 2
bias: False
curv: 1
activation: None
skips: False

In [ ]:
folder_names = set(datasets.keys()) | set(results.keys())
base_dir = os.getcwd()
setup_dirs(folder_names, base_dir)
dump_results(datasets, results, base_dir, run)